In [100]:
import pandas

titanic = pandas.read_csv("kaggle_titanic/train.csv")
test_data = pandas.read_csv("kaggle_titanic/test.csv")

In [102]:
# preprocessing

import re
from sklearn import preprocessing

def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

def get_age_group(age):
    if age < 15:
        return 0
    if age < 50:
        return 1
    return 2

titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
titanic["Embarked"] = titanic["Embarked"].fillna("S")

# new features
titanic["FamilySize"] = titanic["SibSp"] + titanic["Parch"]
titanic["NameLength"] = titanic["Name"].apply(lambda x: len(x))
titanic["Title"] = titanic["Name"].apply(get_title)
# titanic["Age"]  = titanic["Age"].apply(get_age_group)

columns_labels = preprocessing.LabelEncoder()
train_columns_to_encode = ['Sex', 'Embarked', 'Title']

for col in train_columns_to_encode:
    titanic[col] = columns_labels.fit_transform(titanic[col])
    

In [103]:
# features exctractin

from sklearn import cross_validation
from sklearn.feature_selection import SelectKBest, f_classif

predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Embarked", "Title"]

# selector = SelectKBest(f_classif, k=5)
# selector.fit(titanic[predictors], titanic["Survived"])


In [104]:
def class_to_vect(classification):
    cls_count = 2
    vect = []
    for cls in classification:
        v = [0] * cls_count
        v[cls] = 1
        vect.append(v)
    return np.array(vect)

def get_binary_two_outputs(pred_probs):
    results = []
    for p1, p2 in pred_probs:
        if p1 > p2:
            results.append(0)
        else:
            results.append(1)
    return results

Two outputs approach:

In [105]:
# classification and cross validation

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Sequential
from sklearn.cross_validation import KFold
from keras.layers.core import Dense, Dropout, Activation

# alg = LogisticRegression(random_state=1)
# alg = GradientBoostingClassifier()
nn = Sequential()
nn.add(Dense(len(predictors), output_dim=10, activation='tanh'))
nn.add(Dense(10, output_dim=10, activation='tanh'))
nn.add(Dense(10, output_dim=2, activation='softmax'))
nn.compile(loss='mean_squared_error', optimizer='RMSprop')

kf = KFold(titanic.shape[0], n_folds=3, random_state=1)
predictions = []
for train, test in kf:
    train_predictors = titanic[predictors].as_matrix()[train,:]
    target = class_to_vect(titanic.Survived.as_matrix())
    target_train = class_to_vect(titanic.Survived.iloc[train].as_matrix())
    nn.fit(train_predictors, target[train], nb_epoch=20, show_accuracy=True)
    test_predictions = nn.predict_classes(titanic[predictors].as_matrix()[test,:])
    predictions.append(test_predictions)
predictions = np.concatenate(predictions)

# for i in range(10, 200, 10):
#     alg = AdaBoostClassifier(n_estimators=80)
#     alg = KNeighborsClassifier(n_neighbors=i)
#     alg = RandomForestClassifier(random_state=0, n_estimators=i, min_samples_split=4, min_samples_leaf=2)
# scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic["Survived"], cv=3)
# score_means.append(scores.mean())
# # plt.plot(score_means)
# print(score_means)


Epoch 0
594/594 [==============================] - 0s - loss: 0.2622 - acc: 0.4680     
Epoch 1
594/594 [==============================] - 0s - loss: 0.2521 - acc: 0.5791     
Epoch 2
594/594 [==============================] - 0s - loss: 0.2467 - acc: 0.5825     
Epoch 3
594/594 [==============================] - 0s - loss: 0.2426 - acc: 0.5842     
Epoch 4
594/594 [==============================] - 0s - loss: 0.2392 - acc: 0.5926     
Epoch 5
594/594 [==============================] - 0s - loss: 0.2362 - acc: 0.5926     
Epoch 6
594/594 [==============================] - 0s - loss: 0.2336 - acc: 0.5943     
Epoch 7
594/594 [==============================] - 0s - loss: 0.2304 - acc: 0.5943     
Epoch 8
594/594 [==============================] - 0s - loss: 0.2276 - acc: 0.5960     
Epoch 9
594/594 [==============================] - 0s - loss: 0.2251 - acc: 0.6094     
Epoch 10
594/594 [==============================] - 0s - loss: 0.2224 - acc: 0.6397     
Epoch 11
594/594 [=============

Evaluation

In [106]:
def perc_score(actual, prediction):
    error = 0
    N = len(predictions)
    for i in range(N):
        if predictions[i] != actual[i]:
            error += 1
    return float(error) / N

print(perc_score(predictions, titanic.Survived))

0.0


In [109]:
# processing test data 

test_data["Age"] = test_data["Age"].fillna(titanic["Age"].median())
test_data["Embarked"] = test_data["Embarked"].fillna("S")
test_data["Fare"] = test_data["Fare"].fillna(titanic["Fare"].median())

# new features
test_data["FamilySize"] = test_data["SibSp"] + test_data["Parch"]
test_data["NameLength"] = test_data["Name"].apply(lambda x: len(x))
test_data["Title"] = test_data["Name"].apply(get_title)

for col in train_columns_to_encode:
    test_data[col] = columns_labels.fit_transform(test_data[col])


In [111]:
# predict test data

# alg.fit(titanic[predictors], titanic["Survived"])
nn.fit(titanic[predictors].as_matrix(), class_to_vect(titanic.Survived.as_matrix()), nb_epoch=20, show_accuracy=True)
predictions = nn.predict_classes(test_data[predictors].as_matrix())

submission = pandas.DataFrame({
        "PassengerId": test_data["PassengerId"],
        "Survived": predictions
    })
submission.to_csv("kaggle_titanic/submission.csv", index=False)

Epoch 0
891/891 [==============================] - 0s - loss: 0.1382 - acc: 0.8103     
Epoch 1
891/891 [==============================] - 0s - loss: 0.1377 - acc: 0.8114     
Epoch 2
891/891 [==============================] - 0s - loss: 0.1373 - acc: 0.8114     
Epoch 3
891/891 [==============================] - 0s - loss: 0.1377 - acc: 0.8114     
Epoch 4
891/891 [==============================] - 0s - loss: 0.1379 - acc: 0.8036     
Epoch 5
891/891 [==============================] - 0s - loss: 0.1378 - acc: 0.8126     
Epoch 6
891/891 [==============================] - 0s - loss: 0.1375 - acc: 0.8047     
Epoch 7
891/891 [==============================] - 0s - loss: 0.1369 - acc: 0.8103     
Epoch 8
891/891 [==============================] - 0s - loss: 0.1370 - acc: 0.8081     
Epoch 9
891/891 [==============================] - 0s - loss: 0.1368 - acc: 0.8114     
Epoch 10
891/891 [==============================] - 0s - loss: 0.1369 - acc: 0.8081     
Epoch 11
891/891 [=============